Modelling and Evaluation Notebook
Objectives
Answer business requirement 2:
The client is interested to tell whether a given cell contains a malaria parasite or not.
Inputs
inputs/malaria_dataset/cell_images/train
inputs/malaria_dataset/cell_images/test
inputs/malaria_dataset/cell_images/validation
image shape embeddings.
Outputs
Images distribution plot in train, validation, and test set.
Image augmentation.
Class indices to change prediction inference in labels.
Machine learning model creation and training.
Save model.
Learning curve plot for model performance.
Model evaluation on pickle file.
Prediction on the random image file.
Additional Comments | Insights | Conclusions

---

### Import regular packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

---

### Set Working Directory


In [2]:
cwd= os.getcwd()

In [3]:

os.chdir('/workspaces/Malaria-Detection/')
print("You set a new current directory")

You set a new current directory


In [4]:
work_dir = os.getcwd()
work_dir

'/workspaces/Malaria-Detection'

---

### Set input directories

Set train, validation and test paths

In [5]:
my_data_dir = 'inputs/malaria_dataset/cell_images'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

## Set output directory

In [6]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

Old version is already available create a new version.


### Set labels

In [7]:
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/malaria_dataset/cell_images/train'

### Set image shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/v1/image_shape.pkl'

In [9]:
import os

# List the contents of your current directory
print("Contents of current directory:")
print(os.listdir('.'))

# Check if the inputs directory exists
if 'inputs' in os.listdir('.'):
    print("\nContents of inputs directory:")
    print(os.listdir('inputs'))
    
    # Check deeper if malaria_dataset exists
    if 'malaria_dataset' in os.listdir('inputs'):
        print("\nContents of malaria_dataset directory:")
        print(os.listdir('inputs/malaria_dataset'))
        
        # Check if cell_images exists
        if 'cell_images' in os.listdir('inputs/malaria_dataset'):
            print("\nContents of cell_images directory:")
            print(os.listdir('inputs/malaria_dataset/cell_images'))

Contents of current directory:
['outputs', '.git', 'jupyter_notebooks', 'kaggle.json', '_pycache_', 'inputs', 'requirements.txt', 'README.md', 'app_pages']

Contents of inputs directory:
['Uninfected', 'Parasitized', 'validation', 'malaria_dataset']

Contents of malaria_dataset directory:
['cell_images']

Contents of cell_images directory:
['validation']
